In [25]:
# Gihad Abdelhamid
# 20196899
# MMA
# W21
# 869
# 16-Aug-2020

# Answer to Question 7, Part 7a
#Load, clean, and preprocess the data as you find necessary

## 7) 2a - Preprocess the data however you see fit. In code comments, describe what you did and why.

In [72]:
#Import Packages
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score, silhouette_samples

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [76]:
# Read in data from OJ.csv I am assuming that you have the file in the same directory as the py file
df = pd.read_csv("OJ.csv")

In [77]:
# Check some data frame info
df.info()

# Create variables to identify the ID column and the target
Id_col = 'Unnamed: 0'
target_col = 'Purchase'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1070 entries, 0 to 1069
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      1070 non-null   int64  
 1   Purchase        1070 non-null   object 
 2   WeekofPurchase  1070 non-null   int64  
 3   StoreID         1070 non-null   int64  
 4   PriceCH         1070 non-null   float64
 5   PriceMM         1070 non-null   float64
 6   DiscCH          1070 non-null   float64
 7   DiscMM          1070 non-null   float64
 8   SpecialCH       1070 non-null   int64  
 9   SpecialMM       1070 non-null   int64  
 10  LoyalCH         1070 non-null   float64
 11  SalePriceMM     1070 non-null   float64
 12  SalePriceCH     1070 non-null   float64
 13  PriceDiff       1070 non-null   float64
 14  Store7          1070 non-null   object 
 15  PctDiscMM       1070 non-null   float64
 16  PctDiscCH       1070 non-null   float64
 17  ListPriceDiff   1070 non-null   f

In [78]:
# Convert some features into categories since they are not really numeric 
# and will result in inaccuracy issues if they are kept as numeric
df['Purchase']=df['Purchase'].astype('category')
df['StoreID']=df['StoreID'].astype('category')
df['SpecialCH']=df['SpecialCH'].astype('category')
df['SpecialMM']=df['SpecialMM'].astype('category')
df['STORE']=df['STORE'].astype('category')

In [79]:
# Convert Store7 Yes/No into 1/0 flag
df['Store7'] = df.Store7.apply(lambda x: 1 if x =='Yes' else 0)
df['Store7']=df['Store7'].astype('category')

In [80]:
# Check some data frame info after converting to categorical
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1070 entries, 0 to 1069
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   Unnamed: 0      1070 non-null   int64   
 1   Purchase        1070 non-null   category
 2   WeekofPurchase  1070 non-null   int64   
 3   StoreID         1070 non-null   category
 4   PriceCH         1070 non-null   float64 
 5   PriceMM         1070 non-null   float64 
 6   DiscCH          1070 non-null   float64 
 7   DiscMM          1070 non-null   float64 
 8   SpecialCH       1070 non-null   category
 9   SpecialMM       1070 non-null   category
 10  LoyalCH         1070 non-null   float64 
 11  SalePriceMM     1070 non-null   float64 
 12  SalePriceCH     1070 non-null   float64 
 13  PriceDiff       1070 non-null   float64 
 14  Store7          1070 non-null   category
 15  PctDiscMM       1070 non-null   float64 
 16  PctDiscCH       1070 non-null   float64 
 17  ListPriceDiff 

## 7) 2.b Splitting the data

In [81]:
# split the data into 80/20 so that we can do cross validation after training the model on real data, 
# which is the remaining 20%

# I used the train_test_split function to create 4 data frames, training frames for the features and the target 
# and validation frames for the features and target

# The test size parameter defines the split, in this case I used 0.2 since it's an 80/20 split. 

In [82]:
from sklearn.model_selection import train_test_split

X = df.drop([Id_col, target_col], axis=1)
y = df[target_col]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [83]:
# Inspect the dataframes 
X.info()
X.shape
X.head()

X_train.info()
X_train.shape
X_train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1070 entries, 0 to 1069
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   WeekofPurchase  1070 non-null   int64   
 1   StoreID         1070 non-null   category
 2   PriceCH         1070 non-null   float64 
 3   PriceMM         1070 non-null   float64 
 4   DiscCH          1070 non-null   float64 
 5   DiscMM          1070 non-null   float64 
 6   SpecialCH       1070 non-null   category
 7   SpecialMM       1070 non-null   category
 8   LoyalCH         1070 non-null   float64 
 9   SalePriceMM     1070 non-null   float64 
 10  SalePriceCH     1070 non-null   float64 
 11  PriceDiff       1070 non-null   float64 
 12  Store7          1070 non-null   category
 13  PctDiscMM       1070 non-null   float64 
 14  PctDiscCH       1070 non-null   float64 
 15  ListPriceDiff   1070 non-null   float64 
 16  STORE           1070 non-null   category
dtypes: category(5)

(1070, 17)

,WeekofPurchase,StoreID,PriceCH,PriceMM,DiscCH,DiscMM,SpecialCH,SpecialMM,LoyalCH,SalePriceMM,SalePriceCH,PriceDiff,Store7,PctDiscMM,PctDiscCH,ListPriceDiff,STORE
0,237,1,1.75,1.99,0.00,0.0,0,0,0.500000,1.99,1.75,0.24,0,0.000000,0.000000,0.24,1
1,239,1,1.75,1.99,0.00,0.3,0,1,0.600000,1.69,1.75,-0.06,0,0.150754,0.000000,0.24,1
2,245,1,1.86,2.09,0.17,0.0,0,0,0.680000,2.09,1.69,0.40,0,0.000000,0.091398,0.23,1
3,227,1,1.69,1.69,0.00,0.0,0,0,0.400000,1.69,1.69,0.00,0,0.000000,0.000000,0.00,1
4,228,7,1.69,1.69,0.00,0.0,0,0,0.956535,1.69,1.69,0.00,1,0.000000,0.000000,0.00,0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 856 entries, 351 to 860
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   WeekofPurchase  856 non-null    int64   
 1   StoreID         856 non-null    category
 2   PriceCH         856 non-null    float64 
 3   PriceMM         856 non-null    float64 
 4   DiscCH          856 non-null    float64 
 5   DiscMM          856 non-null    float64 
 6   SpecialCH       856 non-null    category
 7   SpecialMM       856 non-null    category
 8   LoyalCH         856 non-null    float64 
 9   SalePriceMM     856 non-null    float64 
 10  SalePriceCH     856 non-null    float64 
 11  PriceDiff       856 non-null    float64 
 12  Store7          856 non-null    category
 13  PctDiscMM       856 non-null    float64 
 14  PctDiscCH       856 non-null    float64 
 15  ListPriceDiff   856 non-null    float64 
 16  STORE           856 non-null    category
dtypes: category(5)

(856, 17)

,WeekofPurchase,StoreID,PriceCH,PriceMM,DiscCH,DiscMM,SpecialCH,SpecialMM,LoyalCH,SalePriceMM,SalePriceCH,PriceDiff,Store7,PctDiscMM,PctDiscCH,ListPriceDiff,STORE
351,278,7,2.06,2.13,0.0,0.0,0,0,0.982408,2.13,2.06,0.07,1,0.0,0.000000,0.07,0
530,236,2,1.75,1.99,0.0,0.0,0,0,0.680000,1.99,1.75,0.24,0,0.0,0.000000,0.24,2
896,259,7,1.86,2.18,0.0,0.0,0,0,0.868928,2.18,1.86,0.32,1,0.0,0.000000,0.32,0
947,260,3,1.99,2.09,0.0,0.0,0,0,0.011649,2.09,1.99,0.10,0,0.0,0.000000,0.10,3
936,231,2,1.69,1.69,0.3,0.0,1,0,0.135607,1.69,1.39,0.30,0,0.0,0.177515,0.00,2


## 7) 2.c Building 3 models

### Naive Bayes

In [86]:
X = df.drop([Id_col, target_col], axis=1)
y = df[target_col]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Naive Bayes algorithm and check the performance metrics
# Naive Bayes doesn't have hyperparameters to tune
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb = gnb.fit(X_train, y_train)
gnb

y_pred_gnb = gnb.predict(X_val)


GaussianNB(priors=None, var_smoothing=1e-09)

In [87]:
# Will create and print a confusion matrix to inspect the performance metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, f1_score

confusion_matrix(y_val, y_pred_gnb)
print(classification_report(y_val, y_pred_gnb))

array([[96, 34],
       [22, 62]], dtype=int64)

              precision    recall  f1-score   support

          CH       0.81      0.74      0.77       130
          MM       0.65      0.74      0.69        84

    accuracy                           0.74       214
   macro avg       0.73      0.74      0.73       214
weighted avg       0.75      0.74      0.74       214



# SVM

In [99]:
# running with different regularization value since this is the main hyper parameter for SVM
from sklearn.svm import SVC

svm_clf = SVC(kernel="linear", C=0.01)
svm_clf.fit(X_train, y_train)

y_pred_svm = svm_clf.predict(X_val)

confusion_matrix(y_val, y_pred_svm)
print(classification_report(y_val, y_pred_svm))

SVC(C=0.01, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

array([[119,  11],
       [ 34,  50]], dtype=int64)

              precision    recall  f1-score   support

          CH       0.78      0.92      0.84       130
          MM       0.82      0.60      0.69        84

    accuracy                           0.79       214
   macro avg       0.80      0.76      0.77       214
weighted avg       0.79      0.79      0.78       214



In [100]:
from sklearn.svm import SVC

svm_clf = SVC(kernel="linear", C=0.1)
svm_clf.fit(X_train, y_train)

y_pred_svm = svm_clf.predict(X_val)

confusion_matrix(y_val, y_pred_svm)
print(classification_report(y_val, y_pred_svm))

SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

array([[115,  15],
       [ 24,  60]], dtype=int64)

              precision    recall  f1-score   support

          CH       0.83      0.88      0.86       130
          MM       0.80      0.71      0.75        84

    accuracy                           0.82       214
   macro avg       0.81      0.80      0.80       214
weighted avg       0.82      0.82      0.82       214



In [96]:
from sklearn.svm import SVC

svm_clf = SVC(kernel="linear", C=1)
svm_clf.fit(X_train, y_train)

y_pred_svm = svm_clf.predict(X_val)

confusion_matrix(y_val, y_pred_svm)
print(classification_report(y_val, y_pred_svm))

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

array([[116,  14],
       [ 25,  59]], dtype=int64)

              precision    recall  f1-score   support

          CH       0.82      0.89      0.86       130
          MM       0.81      0.70      0.75        84

    accuracy                           0.82       214
   macro avg       0.82      0.80      0.80       214
weighted avg       0.82      0.82      0.82       214



In [97]:
from sklearn.svm import SVC

svm_clf = SVC(kernel="linear", C=5)
svm_clf.fit(X_train, y_train)

y_pred_svm = svm_clf.predict(X_val)

confusion_matrix(y_val, y_pred_svm)
print(classification_report(y_val, y_pred_svm))

SVC(C=5, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

array([[116,  14],
       [ 27,  57]], dtype=int64)

              precision    recall  f1-score   support

          CH       0.81      0.89      0.85       130
          MM       0.80      0.68      0.74        84

    accuracy                           0.81       214
   macro avg       0.81      0.79      0.79       214
weighted avg       0.81      0.81      0.80       214



In [98]:
from sklearn.svm import SVC

svm_clf = SVC(kernel="linear", C=15)
svm_clf.fit(X_train, y_train)

y_pred_svm = svm_clf.predict(X_val)

confusion_matrix(y_val, y_pred_svm)
print(classification_report(y_val, y_pred_svm))

SVC(C=15, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

array([[114,  16],
       [ 25,  59]], dtype=int64)

              precision    recall  f1-score   support

          CH       0.82      0.88      0.85       130
          MM       0.79      0.70      0.74        84

    accuracy                           0.81       214
   macro avg       0.80      0.79      0.79       214
weighted avg       0.81      0.81      0.81       214



In [101]:
# Model chosen after trying different values for the hyper parameter. I selected this based on accuracy and precision and recall
from sklearn.svm import SVC

svm_clf = SVC(kernel="linear", C=1)
svm_clf.fit(X_train, y_train)

y_pred_svm = svm_clf.predict(X_val)

confusion_matrix(y_val, y_pred_svm)
print(classification_report(y_val, y_pred_svm))

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

array([[116,  14],
       [ 25,  59]], dtype=int64)

              precision    recall  f1-score   support

          CH       0.82      0.89      0.86       130
          MM       0.81      0.70      0.75        84

    accuracy                           0.82       214
   macro avg       0.82      0.80      0.80       214
weighted avg       0.82      0.82      0.82       214



# XGBoost

In [108]:
# XGBoost has multiple hyper parameters
# n_estimators controls the number of boosting stages
# loss function and will leave that to default
# learning_rate shrinks the contribution of each tree
# max_depth limits the maximum depth of the trees
# max_features which decideds how many features to use and whether we use sqrt or log or an interaction
#random_State just uses a fixed seed

from sklearn.ensemble import GradientBoostingClassifier

clf_gt = GradientBoostingClassifier(
    n_estimators=100, learning_rate=0.05, max_depth=1, 
    random_state=0)
clf_gt.fit(X_train, y_train)

y_pred_gt = clf_gt.predict(X_val)

confusion_matrix(y_val, y_pred_gt)
print(classification_report(y_val, y_pred_gt))

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.05, loss='deviance', max_depth=1,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

array([[116,  14],
       [ 27,  57]], dtype=int64)

              precision    recall  f1-score   support

          CH       0.81      0.89      0.85       130
          MM       0.80      0.68      0.74        84

    accuracy                           0.81       214
   macro avg       0.81      0.79      0.79       214
weighted avg       0.81      0.81      0.80       214



In [109]:
# changing the values of the max_depth hyper parameter
from sklearn.ensemble import GradientBoostingClassifier

clf_gt = GradientBoostingClassifier(
    n_estimators=100, learning_rate=0.05, max_depth=3, 
    random_state=0)
clf_gt.fit(X_train, y_train)

y_pred_gt = clf_gt.predict(X_val)

confusion_matrix(y_val, y_pred_gt)
print(classification_report(y_val, y_pred_gt))

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.05, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

array([[115,  15],
       [ 27,  57]], dtype=int64)

              precision    recall  f1-score   support

          CH       0.81      0.88      0.85       130
          MM       0.79      0.68      0.73        84

    accuracy                           0.80       214
   macro avg       0.80      0.78      0.79       214
weighted avg       0.80      0.80      0.80       214



In [110]:
# changing the values of the max_depth hyper parameter
from sklearn.ensemble import GradientBoostingClassifier

clf_gt = GradientBoostingClassifier(
    n_estimators=100, learning_rate=0.05, max_depth=5, 
    random_state=0)
clf_gt.fit(X_train, y_train)

y_pred_gt = clf_gt.predict(X_val)

confusion_matrix(y_val, y_pred_gt)
print(classification_report(y_val, y_pred_gt))

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.05, loss='deviance', max_depth=5,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

array([[109,  21],
       [ 25,  59]], dtype=int64)

              precision    recall  f1-score   support

          CH       0.81      0.84      0.83       130
          MM       0.74      0.70      0.72        84

    accuracy                           0.79       214
   macro avg       0.78      0.77      0.77       214
weighted avg       0.78      0.79      0.78       214



In [111]:
# changing the values of the n_estimators hyper parameter
from sklearn.ensemble import GradientBoostingClassifier

clf_gt = GradientBoostingClassifier(
    n_estimators=200, learning_rate=0.05, max_depth=1, 
    random_state=0)
clf_gt.fit(X_train, y_train)

y_pred_gt = clf_gt.predict(X_val)

confusion_matrix(y_val, y_pred_gt)
print(classification_report(y_val, y_pred_gt))

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.05, loss='deviance', max_depth=1,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=200,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

array([[115,  15],
       [ 27,  57]], dtype=int64)

              precision    recall  f1-score   support

          CH       0.81      0.88      0.85       130
          MM       0.79      0.68      0.73        84

    accuracy                           0.80       214
   macro avg       0.80      0.78      0.79       214
weighted avg       0.80      0.80      0.80       214



In [112]:
# changing the values of the n_estimators hyper parameter
from sklearn.ensemble import GradientBoostingClassifier

clf_gt = GradientBoostingClassifier(
    n_estimators=50, learning_rate=0.05, max_depth=1, 
    random_state=0)
clf_gt.fit(X_train, y_train)

y_pred_gt = clf_gt.predict(X_val)

confusion_matrix(y_val, y_pred_gt)
print(classification_report(y_val, y_pred_gt))

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.05, loss='deviance', max_depth=1,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=50,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

array([[115,  15],
       [ 29,  55]], dtype=int64)

              precision    recall  f1-score   support

          CH       0.80      0.88      0.84       130
          MM       0.79      0.65      0.71        84

    accuracy                           0.79       214
   macro avg       0.79      0.77      0.78       214
weighted avg       0.79      0.79      0.79       214



In [114]:
# changing the values of the learning_rate hyper parameter
from sklearn.ensemble import GradientBoostingClassifier

clf_gt = GradientBoostingClassifier(
    n_estimators=100, learning_rate=0.1, max_depth=1, 
    random_state=0)
clf_gt.fit(X_train, y_train)

y_pred_gt = clf_gt.predict(X_val)

confusion_matrix(y_val, y_pred_gt)
print(classification_report(y_val, y_pred_gt))

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=1,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

array([[116,  14],
       [ 26,  58]], dtype=int64)

              precision    recall  f1-score   support

          CH       0.82      0.89      0.85       130
          MM       0.81      0.69      0.74        84

    accuracy                           0.81       214
   macro avg       0.81      0.79      0.80       214
weighted avg       0.81      0.81      0.81       214



In [115]:
# changing the values of the learning_rate hyper parameter
from sklearn.ensemble import GradientBoostingClassifier

clf_gt = GradientBoostingClassifier(
    n_estimators=100, learning_rate=0.3, max_depth=1, 
    random_state=0)
clf_gt.fit(X_train, y_train)

y_pred_gt = clf_gt.predict(X_val)

confusion_matrix(y_val, y_pred_gt)
print(classification_report(y_val, y_pred_gt))

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.3, loss='deviance', max_depth=1,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

array([[113,  17],
       [ 27,  57]], dtype=int64)

              precision    recall  f1-score   support

          CH       0.81      0.87      0.84       130
          MM       0.77      0.68      0.72        84

    accuracy                           0.79       214
   macro avg       0.79      0.77      0.78       214
weighted avg       0.79      0.79      0.79       214



In [116]:
#### Chosen model after trying different hyper parameter values
from sklearn.ensemble import GradientBoostingClassifier

clf_gt = GradientBoostingClassifier(
    n_estimators=100, learning_rate=0.1, max_depth=1, 
    random_state=0)
clf_gt.fit(X_train, y_train)

y_pred_gt = clf_gt.predict(X_val)

confusion_matrix(y_val, y_pred_gt)
print(classification_report(y_val, y_pred_gt))

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=1,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

array([[116,  14],
       [ 26,  58]], dtype=int64)

              precision    recall  f1-score   support

          CH       0.82      0.89      0.85       130
          MM       0.81      0.69      0.74        84

    accuracy                           0.81       214
   macro avg       0.81      0.79      0.80       214
weighted avg       0.81      0.81      0.81       214

